# Database Decoder
Returns basic details about any Postgresql database for which you may not have any schema infomation. It lists the non-internal tables, along with their column names, data types, and internal lengths. Each table also shows the total number of rows, and has a sampling of 5 rows of data.

#### Modules

In [2]:
import psycopg2
import pprint
from psycopg2.extensions import AsIs

#### Request database details and create connection and cursor objects

In [ ]:
db_name = input("Database name: ")
db_user = input("Sign-in user: ")
db_pwd = input("User password: ")
conn = psycopg2.connect(database=db_name, user=db_user, password=db_pwd)
cur = conn.cursor()
print(conn)

#### Get list of tables in the database
Internal tables are omitted.<br>
table_names is a tuple of all table names in the database

In [2]:
 # Use information schema to select all public (user-created) tables
cur.execute("""
SELECT table_name 
FROM information_schema.tables 
WHERE table_schema = 'public'
ORDER BY table_name;
""")
table_names = cur.fetchall()

#### Data Type Code Map
Create dictionary containing data type codes mapped to human readable data types. This dictionary is used to list each column's data type as a string rather than the default integer.

In [3]:
cur.execute("SELECT oid, typname FROM pg_catalog.pg_type")
rawtypes = cur.fetchall()
readable_types = {}
    
for type_pair in rawtypes:
    readable_types[type_pair[0]] = type_pair[1]

#### Column Details
Go thru tables and create a nested dictionary containing each table's column names, data types, and internal sizes. <p>Dictionary format:
<pre>
  { 'tbl1': { 'columns': [ { 'internal_size': 4,
                             'name': 'id',
                             'type': 'int4'},
                           { 'internal_size': 4,
                             'name': 'fname',
                             'type': 'str'},  ...
</pre>

In [4]:
db_dict = {}
for tname in table_names:
    column_deets[tname] = {}
    column_deets[tname]["columns"] = [{}]
    
    #get table descriptor data
    sql = cur.mogrify("SELECT * FROM %s LIMIT 0", [AsIs(tname)])
    cur.execute(sql)
    tdesc = cur.description
    for x in range(0, len(tdesc)):
        if x > 0:
            db_dict[tname]["columns"].append({})
        db_dict[tname]["columns"][x]["name"] = tdesc[x][0]
        int_type = tdesc[x][1]
        db_dict[tname]["columns"][x]["type"] = readable_types[int_type]
        db_dict[tname]["columns"][x]["internal_size"] = tdesc[x][3]

#### Table Rows
It's useful to know how large each table is. Add a key to db_dict that holds the number of rows for each table in the dictionary.
<p>Dictionary format:
<pre>
  { 'tbl1': { 'columns': [ { 'internal_size': 4,
                             'name': 'id',
                             'type': 'int4'},
                           { 'internal_size': 4,  ...
              'total': 86529},   ...
</pre>

In [5]:
for key in db_dict:
    sql = cur.mogrify("SELECT COUNT(*) FROM %s", [AsIs(key)])
    cur.execute(sql)
    rowcount=cur.fetchone()[0]
    db_dict[key]["total"] = rowcount

#### Sample Data
It's also nice to see a few examples of the data in the table. Add a key to the dictionary that shows 5 rows from each table.
<p>Dictionary format:
<pre>
  { 'tbl1': { 'columns': [ { 'internal_size': 4,
                             'name': 'id',
                             'type': 'int4'},
                           { 'internal_size': 4,  ...
              'sample_rows': [ ( 1,
                                  datetime.date(2010, 5, 12),
                                  'a string', ... ) ],
              'total': 86529},   ...
</pre>

In [6]:
for tbl in db_dict:
    sql = cur.mogrify("SELECT * FROM %s LIMIT 5", [AsIs(tbl)])
    cur.execute(sql)
    sample_rows = cur.fetchall()
    db_dict[tbl]["sample_rows"] = sample_rows

#### Display Dictionary
The **pprint** module provides **`pprint(db_dictionary)`** to get a tree display. 

In [3]:
import pprint
pp = pprint.PrettyPrinter(indent=2)
pp.pprint(hurricane_info)

conn.close()